### STEP 1: Load AIDev Dataset from HuggingFace

In [33]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [34]:
# Pull requests 
pr = pd.read_parquet("hf://datasets/hao-li/AIDev/all_pull_request.parquet")

# Repository meta 
repo = pd.read_parquet("hf://datasets/hao-li/AIDev/all_repository.parquet")

user = pd.read_parquet("hf://datasets/hao-li/AIDev/all_user.parquet")

In [35]:
pr.columns

Index(['id', 'number', 'title', 'body', 'agent', 'user_id', 'user', 'state',
       'created_at', 'closed_at', 'merged_at', 'repo_id', 'repo_url',
       'html_url'],
      dtype='object')

In [36]:
repo.columns

Index(['id', 'url', 'license', 'full_name', 'language', 'forks', 'stars'], dtype='object')

In [37]:
user.columns

Index(['id', 'login', 'followers', 'following', 'created_at'], dtype='object')

### Step 2: Filter to AI-generated pull requests

In [38]:
df = pr.copy()

# Keep PRs that have a non-null agent label (AI-generated)
df = df[df["agent"].notna()]

# Keep only PRs with a final state
df = df[df["state"].isin(["merged", "closed"])]

# Parse timestamps
df["created_at"] = pd.to_datetime(df["created_at"], errors="coerce")

### Step 3: Merge repository metadata

In [39]:
# pr.repo_id  ↔  repo.id
df = df.merge(
    repo,
    left_on="repo_id",
    right_on="id",
    how="left",
    suffixes=("_pr", "_repo")
)

### Step 4: Define repository groups

In [41]:
# 4.1 Popularity groups based on star ranking

df["stars"] = df["stars"].fillna(0).astype(float)

# Rank by stars (1 = lowest, N = highest), break ties by order of appearance
df["star_rank"] = df["stars"].rank(method="first")

df["pop_group"] = pd.qcut(
    df["star_rank"],
    q=3,
    labels=["Low", "Medium", "High"]
)

In [42]:
# 4.2 Activity groups based on fork ranking
df["forks"] = df["forks"].fillna(0).astype(float)

# Use forks as a proxy for repository activity
df["fork_rank"] = df["forks"].rank(method="first")

df["activity_group"] = pd.qcut(
    df["fork_rank"],
    q=3,
    labels=["Low", "Medium", "High"]
)

### Step 5: Compute agent distributions

In [46]:
# 5.1 Popularity × Agent
pop_agent = (
    df.groupby(["pop_group", "agent"], observed=False)["id"]
      .size()
      .reset_index(name="count")
)

# Convert counts to within-group percentages
pop_agent["pct"] = (
    pop_agent.groupby("pop_group", observed=False)["count"]
             .transform(lambda x: x / x.sum())
)

KeyError: 'Column not found: id'

In [43]:
pop_agent.head()
act_agent.head()


NameError: name 'pop_agent' is not defined